In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Column names
SITE_COL = "Site"
TIME_COL = "Timestamp_Local"
TEMP_COL = "Dry_Bulb_Temperature_C"
GHI_COL = "Global_Horizontal_Radiation_W/m2"
POWER_COL = "Building_Power_kW"
FLAG_COL = "Demand_Response_Flag"
CAPACITY_COL = "Demand_Response_Capacity_kW"

In [2]:
def ensure_output_dir(path: str) -> None:
    """Creates a directory if it doesn't exist."""
    os.makedirs(path, exist_ok=True)

def load_data_with_flags(train_path: str, flags_csv_path: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Load training data and submission CSV with predicted flags
    """
    # Load original training data
    train_df = pd.read_csv(train_path)
    train_df[TIME_COL] = pd.to_datetime(train_df[TIME_COL])
    
    # Load submission with predicted flags
    flags_df = pd.read_csv(flags_csv_path)
    flags_df[TIME_COL] = pd.to_datetime(flags_df[TIME_COL])
    
    print(f"[INFO] Loaded training data: {train_df.shape}")
    print(f"[INFO] Loaded flags data: {flags_df.shape}")
    
    return train_df, flags_df

def prepare_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Prepare features using only the raw columns from the CSV
    """
    df = df.copy()
    df[TIME_COL] = pd.to_datetime(df[TIME_COL])
    
    # Convert numeric columns to proper types
    df[TEMP_COL] = pd.to_numeric(df[TEMP_COL], errors='coerce')
    df[GHI_COL] = pd.to_numeric(df[GHI_COL], errors='coerce')
    df[POWER_COL] = pd.to_numeric(df[POWER_COL], errors='coerce')
    
    # Fill any NaN values with 0
    df[TEMP_COL] = df[TEMP_COL].fillna(0)
    df[GHI_COL] = df[GHI_COL].fillna(0)
    df[POWER_COL] = df[POWER_COL].fillna(0)
    
    return df

In [3]:
class CapacityPredictor(nn.Module):
    """
    Neural network for predicting demand response capacity magnitude
    """
    def __init__(self, input_size: int, hidden_sizes: list = [128, 64, 32]):
        super(CapacityPredictor, self).__init__()
        
        layers = []
        prev_size = input_size
        
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.ReLU(),
                nn.Dropout(0.2)
            ])
            prev_size = hidden_size
        
        # Output layer with positive constraint for magnitude
        layers.append(nn.Linear(prev_size, 1))
        layers.append(nn.Softplus())  # Ensures positive output
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x).squeeze()

In [4]:
def create_data_loader(X: np.ndarray, y: np.ndarray, batch_size: int = 512, shuffle: bool = True) -> DataLoader:
    """Create PyTorch DataLoader from numpy arrays"""
    X_tensor = torch.FloatTensor(X)
    y_tensor = torch.FloatTensor(y)
    dataset = TensorDataset(X_tensor, y_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

def train_capacity_model(X_train: np.ndarray, y_train: np.ndarray, 
                        X_val: np.ndarray, y_val: np.ndarray,
                        epochs: int = 200, lr: float = 0.001, 
                        device: str = "cpu") -> CapacityPredictor:
    """
    Train the capacity prediction neural network
    """
    device = torch.device(device)
    model = CapacityPredictor(X_train.shape[1]).to(device)
    
    # Loss and optimizer
    criterion = nn.SmoothL1Loss()  # Robust to outliers
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5)
    
    # Data loaders
    train_loader = create_data_loader(X_train, y_train, shuffle=True)
    val_loader = create_data_loader(X_val, y_val, shuffle=False)
    
    best_val_loss = float('inf')
    patience_counter = 0
    patience = 15
    
    print("[INFO] Starting training...")
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X = batch_X.to(device)
                batch_y = batch_y.to(device)
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()
        
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        
        # Learning rate scheduling
        scheduler.step(val_loss)
        
        if epoch % 20 == 0:
            print(f"[Epoch {epoch:3d}] Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            # Save best model
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print(f"[INFO] Early stopping at epoch {epoch}")
            break
    
    # Load best model
    model.load_state_dict(best_model_state)
    print(f"[INFO] Training completed. Best validation loss: {best_val_loss:.6f}")
    
    return model

In [5]:
def predict_capacity_pipeline(train_path: str, 
                            flags_csv_path: str,
                            output_path: str,
                            epochs: int = 200,
                            device: str = "cpu"):
    """
    Main pipeline for capacity prediction
    """
    print("[INFO] Starting Capacity Prediction Pipeline")
    print(f"[INFO] Using device: {device}")
    
    # Load data
    train_df, test_with_flags = load_data_with_flags(train_path, flags_csv_path)
    
    # Find capacity column in training data
    capacity_cols = ["Demand_Response_Capacity_kW", "Demand_Response_Capacity_KW"]
    capacity_col = None
    for col in capacity_cols:
        if col in train_df.columns:
            capacity_col = col
            break
    
    if capacity_col is None:
        raise ValueError(f"Capacity column not found. Expected one of: {capacity_cols}")
    
    print(f"[INFO] Using capacity column: {capacity_col}")
    
    # Prepare features for training data
    train_featured = prepare_features(train_df)
    
    # Filter training data to events only (|flag| = 1)
    event_mask = train_featured[FLAG_COL].abs() == 1
    train_events = train_featured[event_mask].copy()
    
    print(f"[INFO] Training events found: {len(train_events)}")
    print(f"[INFO] Flag distribution in events: {train_events[FLAG_COL].value_counts().to_dict()}")
    
    if len(train_events) == 0:
        raise ValueError("No training events found with |flag| = 1")
    
    # Feature columns for model - only raw columns from CSV
    feature_cols = [
        TEMP_COL, GHI_COL, POWER_COL
    ]
    
    # Prepare training data
    X_events = train_events[feature_cols].fillna(0).values
    y_events = train_events[capacity_col].abs().values  # Use absolute capacity as magnitude
    
    # Split for validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_events, y_events, test_size=0.2, random_state=42, shuffle=True
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    print(f"[INFO] Training set: {X_train_scaled.shape}, Validation set: {X_val_scaled.shape}")
    print(f"[INFO] Capacity stats - Mean: {y_train.mean():.2f}, Std: {y_train.std():.2f}")
    
    # Train model
    model = train_capacity_model(
        X_train_scaled, y_train, X_val_scaled, y_val,
        epochs=epochs, device=device
    )
    
    # Evaluate on validation set
    model.eval()
    with torch.no_grad():
        X_val_tensor = torch.FloatTensor(X_val_scaled).to(torch.device(device))
        y_pred_val = model(X_val_tensor).cpu().numpy()
    
    val_mse = mean_squared_error(y_val, y_pred_val)
    val_mae = mean_absolute_error(y_val, y_pred_val)
    print(f"[INFO] Validation MSE: {val_mse:.6f}, MAE: {val_mae:.6f}")
    
    # Prepare test data features
    test_featured = prepare_features(test_with_flags)
    
    # Predict capacity for all test data
    X_test = test_featured[feature_cols].fillna(0).values
    X_test_scaled = scaler.transform(X_test)
    
    model.eval()
    with torch.no_grad():
        X_test_tensor = torch.FloatTensor(X_test_scaled).to(torch.device(device))
        capacity_magnitude = model(X_test_tensor).cpu().numpy()
    
    # Apply sign based on flag and set capacity
    flags = test_featured[FLAG_COL].values
    capacity_pred = np.zeros(len(flags))
    
    # For non-zero flags, apply predicted magnitude with flag sign
    non_zero_mask = flags != 0
    capacity_pred[non_zero_mask] = np.sign(flags[non_zero_mask]) * capacity_magnitude[non_zero_mask]
    
    # Create final submission
    submission = pd.DataFrame({
        SITE_COL: test_featured[SITE_COL].values,
        TIME_COL: test_featured[TIME_COL].dt.strftime("%Y-%m-%d %H:%M:%S"),
        FLAG_COL: flags,
        CAPACITY_COL: capacity_pred
    })
    
    # Save submission
    ensure_output_dir(os.path.dirname(output_path))
    submission.to_csv(output_path, index=False)
    
    print(f"[OK] Capacity predictions saved to: {output_path}")
    print(f"[INFO] Submission shape: {submission.shape}")
    print(f"[INFO] Capacity distribution:")
    print(f"  - Zero capacity (flag=0): {np.sum(capacity_pred == 0)}")
    print(f"  - Positive capacity (flag=1): {np.sum(capacity_pred > 0)}")
    print(f"  - Negative capacity (flag=-1): {np.sum(capacity_pred < 0)}")
    print(f"  - Capacity stats (non-zero): Mean={capacity_pred[capacity_pred!=0].mean():.3f}, Std={capacity_pred[capacity_pred!=0].std():.3f}")
    
    return model, scaler, submission

In [19]:
test_data = pd.read_csv("/Users/ibrahimyucel/Downloads/ULUSLARARASI_ENERJI_YARISMASI/data2/test-data-v0.2.csv")
submission = pd.read_csv("/Users/ibrahimyucel/Downloads/ULUSLARARASI_ENERJI_YARISMASI/codes/phase_2_codes/outputs_simple/xgb_submission_optimized.csv")

# Convert timestamps
test_data["Timestamp_Local"] = pd.to_datetime(test_data["Timestamp_Local"])
submission["Timestamp_Local"] = pd.to_datetime(submission["Timestamp_Local"])

# Merge on Site and Timestamp_Local
merged_data = test_data.merge(
    submission[["Site", "Timestamp_Local", "Demand_Response_Flag"]], 
    on=["Site", "Timestamp_Local"], 
    how="left"
)

# Save merged file
merged_data.to_csv("/Users/ibrahimyucel/Downloads/ULUSLARARASI_ENERJI_YARISMASI/codes/phase_2_codes/outputs_simple/test_optimized_xgboost_merged_data_with_flags.csv", index=False)

print("Merged file created!")
print(f"Shape: {merged_data.shape}")
print(f"Columns: {list(merged_data.columns)}")

Merged file created!
Shape: (105120, 7)
Columns: ['Site', 'Timestamp_Local', 'Dry_Bulb_Temperature_C', 'Global_Horizontal_Radiation_W/m2', 'Building_Power_kW', 'Demand_Response_Flag_x', 'Demand_Response_Flag_y']


In [20]:
merged_data.drop(columns=["Demand_Response_Flag_x"], inplace=True)
merged_data.rename(columns={"Demand_Response_Flag_y": "Demand_Response_Flag"}, inplace=True)
merged_data.to_csv("/Users/ibrahimyucel/Downloads/ULUSLARARASI_ENERJI_YARISMASI/codes/phase_2_codes/outputs_simple/33test_optimized_xgboost_merged_data_with_flags.csv", index=False)

In [21]:
merged_data.to_csv("/Users/ibrahimyucel/Downloads/ULUSLARARASI_ENERJI_YARISMASI/codes/phase_2_codes/outputs_simple/33test_optimized_xgboost_merged_data_with_flags.csv", index=False)

In [22]:
if __name__ == "__main__":
    # Configuration
    TRAIN_PATH = "/Users/ibrahimyucel/Downloads/ULUSLARARASI_ENERJI_YARISMASI/data2/training-data-v0.2.csv"
    
    # You can use any of the generated submission files
    FLAGS_CSV_PATH = "/Users/ibrahimyucel/Downloads/ULUSLARARASI_ENERJI_YARISMASI/codes/phase_2_codes/outputs_simple/33test_optimized_xgboost_merged_data_with_flags.csv"
    
    OUTPUT_PATH = "/Users/ibrahimyucel/Downloads/ULUSLARARASI_ENERJI_YARISMASI/codes/phase_2_codes/outputs_simple/capacity_predictions_nn4.csv"
    
    # Run pipeline
    model, scaler, submission = predict_capacity_pipeline(
        train_path=TRAIN_PATH,
        flags_csv_path=FLAGS_CSV_PATH,
        output_path=OUTPUT_PATH,
        epochs=200,
        device="cpu"  # Change to "cuda" if you have GPU
    )
    
    print("[DONE] Capacity prediction pipeline completed!") 

[INFO] Starting Capacity Prediction Pipeline
[INFO] Using device: cpu
[INFO] Loaded training data: (105120, 7)
[INFO] Loaded flags data: (105120, 6)
[INFO] Using capacity column: Demand_Response_Capacity_kW
[INFO] Training events found: 3109
[INFO] Flag distribution in events: {-1: 2262, 1: 847}
[INFO] Training set: (2487, 3), Validation set: (622, 3)
[INFO] Capacity stats - Mean: 12.29, Std: 22.08
[INFO] Starting training...
[Epoch   0] Train Loss: 11.448710, Val Loss: 12.078916
[Epoch  20] Train Loss: 8.525351, Val Loss: 8.865434
[Epoch  40] Train Loss: 8.081026, Val Loss: 8.277314
[Epoch  60] Train Loss: 7.973694, Val Loss: 7.944939
[Epoch  80] Train Loss: 7.752458, Val Loss: 7.717909
[Epoch 100] Train Loss: 7.456670, Val Loss: 7.598409
[Epoch 120] Train Loss: 7.440469, Val Loss: 7.555068
[INFO] Early stopping at epoch 133
[INFO] Training completed. Best validation loss: 7.491379
[INFO] Validation MSE: 343.180018, MAE: 8.006022
[OK] Capacity predictions saved to: /Users/ibrahimyucel